In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [2]:
database_username = 'galeras'
database_password = 'galeras234'
database_ip       = '192.168.0.1'
database_name     = 'galeras'
database_port     = '6603' # replace with your port
database_connection = create_engine('mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_port, database_name))


In [45]:
sql = "SELECT * FROM code_generation_from_docstring_dataset"
df = pd.read_sql_query(sql, database_connection)

In [46]:
df.head()

,id,commit_id,repo,path,file_name,fun_name,commit_message,code,url,language,...,nloc,token_counts,n_ast_nodes,n_identifiers,d_id,docstring,dn_words,d_vocab_size,dn_whitespaces,d_language
0,232037,43e3a4011080911901176aab919c0ecf5046ddd3,plotly.py,packages/python/plotly/plotly/graph_objs/layou...,_layer.py,minzoom,switch to black .22,def minzoom(self):\n \n return s...,https://github.com/plotly/plotly.py.git,Python,...,2,11,22,2,63481,\n Sets the minimum zoom level (mapbox....,42,37,101,en
1,196771,f757f3daae6e11ea0cfb7dadc133274d8d74315f,sympy,sympy/concrete/summations.py,summations.py,telescopic,Reordered imports 2,"def telescopic(L, R, limits):\n \n (i, a...",https://github.com/sympy/sympy.git,Python,...,27,242,374,27,48161,\n Tries to perform the summation using the...,14,13,24,en
2,230902,43e3a4011080911901176aab919c0ecf5046ddd3,plotly.py,packages/python/plotly/plotly/graph_objs/layou...,_annotation.py,startarrowsize,switch to black .22,def startarrowsize(self):\n \n r...,https://github.com/plotly/plotly.py.git,Python,...,2,11,22,2,62575,\n Sets the size of the start annotatio...,51,45,117,en
3,266879,8b2e6285650ec42ec4a19075a8567047e8304ba2,ansible,lib/ansible/galaxy/dependency_resolution/provi...,providers.py,get_dependencies,galaxy - Clean up type hints and imports.,"def get_dependencies(self, candidate):\n ...",https://github.com/ansible/ansible.git,Python,...,13,60,115,12,78638,Get direct dependencies of a candidate.\n\n ...,18,17,49,en
4,159564,e798bf049f036a5865c14d4343ed8a833864aabe,rasa,rasa/shared/core/trackers.py,trackers.py,active_loop_name,convert TrackerActiveLoop to a dataclass,def active_loop_name(self) -> Optional[Text]:\...,https://github.com/RasaHQ/rasa.git,Python,...,8,33,54,7,38336,Get the name of the currently active loop.\n\n...,22,13,36,en


In [37]:
json_file="/workspaces/chat-gpt-failures/datasets/galeras_se_tasks_dataset/code_completion_dataset_3k.json"
df.to_json(json_file, orient='records')

In [47]:
import json
json_str = df.to_json(orient='records')
json_obj = json.loads(json_str)


In [48]:
for obj in json_obj:
    documentation = {}
    documentation["docstring"] = obj["docstring"]
    del obj["docstring"]
    documentation["n_words"] = obj["dn_words"]
    del obj["dn_words"]
    documentation["vocab_size"] = obj["d_vocab_size"]
    del obj["d_vocab_size"]
    documentation["n_whitespaces"] = obj["dn_whitespaces"]
    del obj["dn_whitespaces"]
    documentation["language"]=obj["d_language"]
    del obj["d_language"]
    obj["documentation"] = documentation
    

In [49]:
json_str = json.dumps(json_obj)


In [50]:
json_df = pd.read_json(json_str)


In [51]:
json_df

,id,commit_id,repo,path,file_name,fun_name,commit_message,code,url,language,...,n_whitespaces,n_words,vocab_size,complexity,nloc,token_counts,n_ast_nodes,n_identifiers,d_id,documentation
0,232037,43e3a4011080911901176aab919c0ecf5046ddd3,plotly.py,packages/python/plotly/plotly/graph_objs/layou...,_layer.py,minzoom,switch to black .22,def minzoom(self):\n \n return s...,https://github.com/plotly/plotly.py.git,Python,...,18,4,4,1,2,11,22,2,63481,{'docstring': ' Sets the minimum zoom ...
1,196771,f757f3daae6e11ea0cfb7dadc133274d8d74315f,sympy,sympy/concrete/summations.py,summations.py,telescopic,Reordered imports 2,"def telescopic(L, R, limits):\n \n (i, a...",https://github.com/sympy/sympy.git,Python,...,391,189,104,16,27,242,374,27,48161,{'docstring': ' Tries to perform the summa...
2,230902,43e3a4011080911901176aab919c0ecf5046ddd3,plotly.py,packages/python/plotly/plotly/graph_objs/layou...,_annotation.py,startarrowsize,switch to black .22,def startarrowsize(self):\n \n r...,https://github.com/plotly/plotly.py.git,Python,...,18,4,4,1,2,11,22,2,62575,{'docstring': ' Sets the size of the s...
3,266879,8b2e6285650ec42ec4a19075a8567047e8304ba2,ansible,lib/ansible/galaxy/dependency_resolution/provi...,providers.py,get_dependencies,galaxy - Clean up type hints and imports.,"def get_dependencies(self, candidate):\n ...",https://github.com/ansible/ansible.git,Python,...,364,178,125,4,13,60,115,12,78638,{'docstring': 'Get direct dependencies of a ca...
4,159564,e798bf049f036a5865c14d4343ed8a833864aabe,rasa,rasa/shared/core/trackers.py,trackers.py,active_loop_name,convert TrackerActiveLoop to a dataclass,def active_loop_name(self) -> Optional[Text]:\...,https://github.com/RasaHQ/rasa.git,Python,...,47,15,13,3,8,33,54,7,38336,{'docstring': 'Get the name of the currently a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,322093,b0c35d5e1ff02a634fa26392b60d3885c2c78677,PaddleNLP,paddlenlp/transformers/blenderbot_small/modeli...,modeling.py,forward,Fix the attention mask for fp16 (#1585),"def forward(self, input_ids=None, attention_ma...",https://github.com/PaddlePaddle/PaddleNLP.git,Python,...,188,55,38,3,15,123,196,25,118053,{'docstring': ' Returns: T...
2996,204594,9c19aff7c7561e3a82978a272ecdaad40dda5c00,django,django/core/management/base.py,base.py,no_translations,Refs #33476 -- Reformatted code with Black.,def no_translations(handle_func):\n \n,https://github.com/django/django.git,Python,...,5,2,2,1,3,10,13,2,50805,{'docstring': 'Decorator that forces a command...
2997,203580,9c19aff7c7561e3a82978a272ecdaad40dda5c00,django,django/contrib/admindocs/views.py,views.py,get_return_data_type,Refs #33476 -- Reformatted code with Black.,def get_return_data_type(func_name):\n \n ...,https://github.com/django/django.git,Python,...,59,14,11,4,7,36,73,4,50457,{'docstring': 'Return a somewhat-helpful data ...
2998,122864,b8ae8e3fa10f9abe998459fac1513915acee776d,jax,tests/filecheck/jax_filecheck_helpers.py,jax_filecheck_helpers.py,print_ir,(NFC) Prepare for migration from producing MHL...,def print_ir(*prototypes):\n def lower(f):\n ...,https://github.com/google/jax.git,Python,...,84,35,31,1,3,10,198,27,27259,{'docstring': 'Prints the MLIR IR that results...


In [52]:
json_file="/workspaces/chat-gpt-failures/datasets/galeras_se_tasks_dataset/code_generation_from_docstring_dataset_3k.json"
json_df.to_json(json_file, orient='records')